In [ ]:
# Import required packages
import sys
# !{sys.executable} -m pip install seaborn
from glob import glob
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from shutil import copy,move
from distutils.dir_util import copy_tree


In [ ]:
files = sorted(glob('./5/*/*results.json'))
baseline = "../BSE-calc-test/5qubit/A2_HWE-CNOT/A2_HWE-CNOT_results.json"

In [ ]:
glob('5/*/*_results.json')

In [ ]:

quantum_results = []
quantum_data = []
for i in files:
    print(i)
    with open(i,'r') as f:
        data = json.load(f)
    ratio = os.path.basename(os.path.dirname(i))
    print(ratio)
    quantum_data.append((ratio,data['R2_train'],data['R2_test']))
    quantum_results.append(('A2_HWE-CNOT',ratio,'R2_train',data['R2_train']))
    quantum_results.append(("A2_HWE-CNOT",ratio,'R2_test',data['R2_test']))

quantum_data = np.array(quantum_data).astype(float)

In [ ]:
classical=pd.read_excel("../classical_testing/BSE/PCA5_classical.xlsx",index_col=0)

In [ ]:
classical.melt(id_vars=["index","size"],value_vars=["R2_train","R2_test"])

In [ ]:
q_df=pd.DataFrame(quantum_results,columns=["index","size","variable","value"])
q_df=q_df.astype({'size':float,"value":float})
c_df=classical.melt(id_vars=["index","size"],value_vars=["R2_train","R2_test"])
c_df=c_df.astype({'size':float,"value":float})

In [ ]:
melted=pd.concat([c_df,q_df])

In [ ]:
palette=sns.color_palette('Paired',10)
model_pal = dict(zip(set(melted['index']),palette))


SMALL_SIZE = 8
MEDIUM_SIZE = 12
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
fig = plt.figure(figsize=(8,5))

ax = sns.lineplot(data=melted,x='size',y='value',hue="index",style='variable',palette=model_pal, markers=True)
ax.legend_.remove()





# # Add separate legend for hue (category)
handles, labels = ax.get_legend_handles_labels()
hue_legend = fig.legend(handles[1:11]+handles[12:], labels[1:11]+labels[12:], title='Model/Dataset', bbox_to_anchor=(1.55, 0.9), borderaxespad=0,ncols=3)

# Add separate legend for style
# style_legend = plt.legend(handles[12:], labels[12:], bbox_to_anchor=(1, 0.2), loc='upper left', borderaxespad=0.5)

# Add the hue legend back so both are displayed
plt.gca().add_artist(hue_legend)


ax.set_ylim(0,1.05)
ax.set_xlim(0.05,0.85)
ax.set_ylabel('R$^{2}$')
ax.set_xlabel('Training Set Ratio')
plt.tight_layout()
plt.savefig('../images/BSE5_learning_curves.png', bbox_inches='tight', pad_inches=0.1)
plt.show()